In [ ]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import datetime as dt

import re, string
import nltk
import contractions
from nltk.corpus import stopwords

from api_keys import api_key

In [ ]:
def request_song_info(artist_id, page_number):
    base_url = f'https://api.genius.com'
    headers = {f'Authorization': f'Bearer {api_key}'}
    search_url = f'{base_url}/artists/{artist_id}/songs?per_page=40&page={page_number}&sort=popularity'
    response = requests.get(search_url, headers=headers)

    return response

In [ ]:
jay_dict = {}
beyonce_dict = {}
destiny_dict = {}
artist_dict = {2 : jay_dict, 498 : beyonce_dict, 2831 : destiny_dict}
        
for artist_id, artist_dict in artist_dict.items():      

    for page in range (1, 20):
        response = request_song_info(artist_id, page)
        json = response.json()
        remote_song_info = None

        for song in json['response']['songs']:
            if str(artist_id) in str(song['primary_artist']['id']):
                remote_song_info = song

                if remote_song_info:
                    song_url = remote_song_info['url']

                if len(artist_dict) == 400:
                    break

                else:
                    artist_dict.update({song['title'].lower() : song_url})

In [ ]:
print(f'the jay_z_dict contains {len(jay_dict)} songs.')
print(f'the beyonce_dict contains {len(beyonce_dict)} songs.')
print(f'the destiny_dict contains {len(destiny_dict)} songs.')

In [ ]:
def scrap_song_url(song_url, date_index):
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, 'html.parser')
    release_date = dt.datetime.strptime(html.find_all('span', {'class': 'metadata_unit-info metadata_unit-info--text_only'})[date_index].get_text(), '%B %d, %Y').strftime('%Y%m%d')
    lyrics = html.find('div', class_='lyrics').get_text()

    return release_date, lyrics

In [ ]:
relationship_start = scrap_song_url(jay_dict["'03 bonnie & clyde"], 1)[0]
print(int(relationship_start))

In [ ]:
pre_jay_dict = {}
post_jay_dict = {}

for song, song_url in jay_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_jay_dict) == 50:
                continue
            else:
                post_jay_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})
        else:
            if len(pre_jay_dict) == 50:
                continue
            else:
                pre_jay_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_jay_dict) == 50:
                    continue
                else:
                    post_jay_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
            else:
                if len(pre_jay_dict) == 50:
                    continue
                else:
                    pre_jay_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})                
                
        except:
            print(f'FAILED on {song}')

In [ ]:
print(len(pre_jay_dict))
print(len(post_jay_dict))

In [ ]:
pre_beyonce_dict = {}
post_beyonce_dict = {}

for song, song_url in beyonce_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_beyonce_dict) == 50:
                continue
            else:
                post_beyonce_dict.update({song : [scrap_song_url(song_url, 1)[0],  song_url, scrap_song_url(song_url, 1)[1]]})
        else:
            if len(pre_beyonce_dict) == 50:
                continue
            else:
                pre_beyonce_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_beyonce_dict) == 50:
                    continue
                else:
                    post_beyonce_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
            else:
                if len(pre_beyonce_dict) == 50:
                    continue
                else:
                    pre_beyonce_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})                
                
        except:
            print(f'FAILED on {song}')

In [ ]:
print(len(pre_beyonce_dict))
print(len(post_beyonce_dict))

In [ ]:
pre_destiny_dict = {}

for song, song_url in destiny_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) < int(relationship_start):
            if len(pre_destiny_dict) == 50:
                continue
            else:
                pre_destiny_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})
    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) < int(relationship_start):
                if len(pre_destiny_dict) == 50:
                    continue
                else:
                    pre_destiny_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
        except:
            print(f'FAILED on {song}')

In [ ]:
print(len(pre_destiny_dict))

In [ ]:
pre_jay_df = pd.DataFrame.from_dict(pre_jay_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics', ])
post_jay_df = pd.DataFrame.from_dict(post_jay_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
pre_beyonce_df = pd.DataFrame.from_dict(pre_beyonce_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
post_beyonce_df = pd.DataFrame.from_dict(post_beyonce_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
pre_destiny_df = pd.DataFrame.from_dict(pre_destiny_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])

In [ ]:
stop = stopwords.words('english')

def clean_text(artist_df):
    artist_df = artist_df.apply(lambda x: x.replace('\n', ' '))
    artist_df = artist_df.apply(lambda x: re.sub('\[[^]]*\]', '', x))
    artist_df = artist_df.apply(lambda x: x.lower())
    artist_df = artist_df.apply(lambda x: contractions.fix(x))
    artist_df = artist_df.apply(lambda x: re.sub(r'[^\w\s]','', x))
    artist_df = artist_df.apply(lambda x: [item for item in x.split() if item not in stop])
    return artist_df

In [ ]:
pre_jay_df.lyrics = clean_text(pre_jay_df.lyrics)
post_jay_df.lyrics = clean_text(post_jay_df.lyrics)
pre_beyonce_df.lyrics = clean_text(pre_beyonce_df.lyrics)
post_beyonce_df.lyrics = clean_text(post_beyonce_df.lyrics)
pre_destiny_df.lyrics = clean_text(pre_destiny_df.lyrics)

In [ ]:
pre_beyonce_df = pd.concat([pre_beyonce_df, pre_destiny_df])

In [ ]:
pre_jay_df['unique_words'] = pre_jay_df.lyrics.apply(lambda x: len(set(x)))
pre_jay_df['word_count'] = pre_jay_df.lyrics.apply(lambda x: len(x))
pre_jay_df['percent_unique'] = pre_jay_df.unique_words / pre_jay_df.word_count

post_jay_df['unique_words'] = post_jay_df.lyrics.apply(lambda x: len(set(x)))
post_jay_df['word_count'] = post_jay_df.lyrics.apply(lambda x: len(x))
post_jay_df['percent_unique'] = post_jay_df.unique_words / post_jay_df.word_count

pre_beyonce_df['unique_words'] = pre_beyonce_df.lyrics.apply(lambda x: len(set(x)))
pre_beyonce_df['word_count'] = pre_beyonce_df.lyrics.apply(lambda x: len(x))
pre_beyonce_df['percent_unique'] = pre_beyonce_df.unique_words / pre_beyonce_df.word_count

post_beyonce_df['unique_words'] = post_beyonce_df.lyrics.apply(lambda x: len(set(x)))
post_beyonce_df['word_count'] = post_beyonce_df.lyrics.apply(lambda x: len(x))
post_beyonce_df['percent_unique'] = post_beyonce_df.unique_words / post_beyonce_df.word_count

In [ ]:
pre_jay_df['years'] = pd.to_datetime(pre_jay_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
pre_jay_df['years'] = pre_jay_df['years'].dt.days / 365

post_jay_df['years'] = pd.to_datetime(post_jay_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
post_jay_df['years'] = post_jay_df['years'].dt.days / 365

pre_beyonce_df['years'] = pd.to_datetime(pre_beyonce_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
pre_beyonce_df['years'] = pre_beyonce_df['years'].dt.days / 365

post_beyonce_df['years'] = pd.to_datetime(post_beyonce_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
post_beyonce_df['years'] = post_beyonce_df['years'].dt.days / 365

In [ ]:
lyric_data = {'sj_years' : pre_jay_df.years.values.tolist(), 
               'sj_percent_unique' : pre_jay_df.percent_unique.values.tolist(),
               'sj_word_count' : pre_jay_df.word_count.values.tolist(),
               'sj_song': pre_jay_df.index.tolist(),
               'sj_song_url': pre_jay_df.song_url.values.tolist(),
                'mj_years' : post_jay_df.years.values.tolist(), 
               'mj_percent_unique' : post_jay_df.percent_unique.values.tolist(),
               'mj_word_count' : post_jay_df.word_count.values.tolist(),
               'mj_song': post_jay_df.index.tolist(),
               'mj_song_url': post_jay_df.song_url.values.tolist(),
              'sb_years' : pre_beyonce_df.years.values.tolist(), 
               'sb_percent_unique' : pre_beyonce_df.percent_unique.values.tolist(),
               'sb_word_count' : pre_beyonce_df.word_count.values.tolist(),
               'sb_song': pre_beyonce_df.index.tolist(),
               'sb_song_url': pre_beyonce_df.song_url.values.tolist(),
                'mb_years' : post_beyonce_df.years.values.tolist(), 
               'mb_percent_unique' : post_beyonce_df.percent_unique.values.tolist(),
               'mb_word_count' : post_beyonce_df.word_count.values.tolist(),
               'mb_song': post_beyonce_df.index.tolist(),
               'mb_song_url': post_beyonce_df.song_url.values.tolist()}

In [ ]:
lyric_data